## Attacking Apples neural hash

This notebook will describe a whitebox attack on the underlying neural network of apples neural hash.

### Loading the model:
Source: https://github.com/AsuharietYgvar/AppleNeuralHash2ONNX

In [21]:
import onnxruntime
import numpy as np
from PIL import Image

In [22]:
MODEL_PATH = "./model.onnx"
OUTPUT_MATRIX_PATH = "./neuralhash_128x96_seed1.dat"

In [23]:
def neural_hash(image):
    # Load ONNX model
    session = onnxruntime.InferenceSession(MODEL_PATH)

    # Load output hash matrix
    seed1 = open(OUTPUT_MATRIX_PATH, 'rb').read()[128:]
    seed1 = np.frombuffer(seed1, dtype=np.float32)
    seed1 = seed1.reshape([96, 128])

    # Preprocess image
    image = Image.open(image).convert('RGB')
    image = image.resize([360, 360])
    arr = np.array(image).astype(np.float32) / 255.0
    arr = arr * 2.0 - 1.0
    arr = arr.transpose(2, 0, 1).reshape([1, 3, 360, 360])

    # Run model
    inputs = {session.get_inputs()[0].name: arr}
    outs = session.run(None, inputs)

    # Convert model output to hex hash
    hash_output = seed1.dot(outs[0].flatten())
    hash_bits = ''.join(['1' if it >= 0 else '0' for it in hash_output])
    hash_hex = '{:0{}x}'.format(int(hash_bits, 2), len(hash_bits) // 4)

    return hash_hex

In [24]:
neural_hash("../shrek.png")

'55d07b72427978ac7a8fd1d9'

### Producing hash collisions
Source: https://github.com/anishathalye/neural-hash-collider

In [27]:
import submodule.collisions as collisions

ModuleNotFoundError: No module named 'util'

In [28]:
collisions.collide("../../shrek.png", iterations=100, target="55d07b72427978ac7a8fd100")

NameError: name 'collisions' is not defined